ME975 Satellite Data Assimilation and Analysis - Assignment 1

Jacob Currie - 201718558

Below:
**Initial setup script**, installs geemap (google maps style mapping/plotting)
Prompts earthengine authentication token
imports relevant libraries
initialises earth engine

Only needs to be run once at the start of each session!

In [ ]:
#Inital Setup
!pip install geemap
!earthengine authenticate 
import ee
import geemap.eefolium
import pprint as pp
import matplotlib.pyplot as plt
ee.Initialize()

Below:
The **Main Script** using earth engine to collect, process and store the relevant data.

It only needs to be run once to acquire and store the data for use elsewhere.

(This takes time to run, the .getInfo() method causes delay in acquiring data as it pulled from GEE servers)

In [ ]:
#Main Script
DistrictNamesList = ['Borders', 'Central', 'Dumfries and Gal', 'Fyfe', 'Grampian', 'Highland', 'Lothian', 'Orkney', 'Shetland Islands', 'Strathclyde', 'Tayside', 'Western Isles'] #List of the level2 District Names

NO2_2019List = [] #Empty lists for storing per-district data
NO2_2020List = []
NO2_2021List = []
PopDensList = []

Scotland_Bounds = ee.FeatureCollection("FAO/GAUL/2015/level2").filterMetadata('ADM1_NAME', 'equals', 'Scotland') #Getting Scotland District Bounds
Population_Data = ee.Image("JRC/GHSL/P2016/POP_GPW_GLOBE_V1/2015").select('population_count').clip(Scotland_Bounds.geometry()) #Getting Population data
NO2_Data = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_NO2").filterBounds(Scotland_Bounds.geometry()) #Getting total NO2 data

NO2_2019 = NO2_Data.filterDate('2019-01-01', '2019-12-31').mean().select(['tropospheric_NO2_column_number_density']).clip(Scotland_Bounds.geometry()) #Per-Year mean NO2 column density image clipped to Scotland
NO2_2020 = NO2_Data.filterDate('2020-01-01', '2020-12-31').mean().select(['tropospheric_NO2_column_number_density']).clip(Scotland_Bounds.geometry())
NO2_2021 = NO2_Data.filterDate('2021-01-01', '2021-12-31').mean().select(['tropospheric_NO2_column_number_density']).clip(Scotland_Bounds.geometry()) 

for District in DistrictNamesList: #For every district - calling .getInfo() inside loop - inefficient but it works
  District_Bounds = Scotland_Bounds.filterMetadata('ADM2_NAME', 'equals', District).geometry() #Getting level2 district data
  NO2_2019List.append(NO2_2019.reduceRegion(**{'geometry': District_Bounds, 'reducer': ee.Reducer.mean(),'scale': 1000,}).getInfo()['tropospheric_NO2_column_number_density']) #NO2 value for each district
  NO2_2020List.append(NO2_2020.reduceRegion(**{'geometry': District_Bounds, 'reducer': ee.Reducer.mean(),'scale': 1000,}).getInfo()['tropospheric_NO2_column_number_density']) #per-year NO2 value (reduce over region)
  NO2_2021List.append(NO2_2021.reduceRegion(**{'geometry': District_Bounds, 'reducer': ee.Reducer.mean(),'scale': 1000,}).getInfo()['tropospheric_NO2_column_number_density'])
  PopSize = Population_Data.reduceRegion(**{'geometry': District_Bounds, 'reducer': ee.Reducer.sum(), 'scale': 250, 'maxPixels':1e9})  #Getting total population size per district (reduce over region)
  PopDensList.append(PopSize.getInfo()['population_count']/ District_Bounds.area().getInfo()) #Calculating population density per district {=total population / district area}

for i in range(len(DistrictNamesList)): #PRINTING RESULTS TO CONSOLE - for each district - printing NO2 and population density
  print('District: '+DistrictNamesList[i])
  print('(mol/m^2) 2019 NO2: '+str(round(NO2_2019List[i],12))+' //2020 NO2: '+str(round(NO2_2020List[i],12))+' //2021 NO2: '+str(round(NO2_2021List[i],12))+' //Population Density: '+str(PopDensList[i]))
  print('---------------------')


Below: The **Mapping Script** that produces the Geemap (Google maps style) figures, each different set of data (year selection, NO2, population size, etc.) are represented as different layers on the map that can be shown and hidden from the layer menu.

In [ ]:
#Geemap Mapping Script
#Some lines may be commented in and out to view only certain results/legends - Colab wont let Map go fullscreen
#plotting options
NO2_viz = {'min': 0, 'max': 0.00004, 'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']}
Pop_viz = {'min': 0, 'max': 6, 'palette': ['white', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']}
colors = ['red', 'green', 'blue', 'pink', 'black', 'white', 'orange', 'yellow', 'purple', 'gray', 'brown', 'lime']

#Making Geemap + layers
Map = geemap.Map(center=[57.404790, -4.304002], zoom=6) #creating map
Map.addLayer(Population_Data, Pop_viz, 'Population')  #population map
Map.addLayer(NO2_2019, NO2_viz, 'NO2 2019')
Map.addLayer(NO2_2020, NO2_viz, 'NO2 2020') #No2 maps
Map.addLayer(NO2_2021, NO2_viz, 'NO2 2021')
Map.addLayer(Scotland_Bounds, {}, 'Bounds') #district bounds
Map.addLayerControl()

import matplotlib.colors as cl #color to hexadecimal method needed from library for legend dictionaries

boundslegend_dict = {} #Creating district bounds legend with colours
for i in range(len(DistrictNamesList)):
  boundslegend_dict[DistrictNamesList[i]] = cl.to_hex(colors[i])

#creating NO2 legend dictionary with colours
no2legend_dict = {'0':cl.to_hex('black'), '0.66e-5':cl.to_hex('blue'), '1.33e-5':cl.to_hex('purple'), '2.00e-5':cl.to_hex('cyan'), '2.66e-5':cl.to_hex('green'), '3.33e-5':cl.to_hex('yellow'), '4.00e-5':cl.to_hex('red')}
#creating population legend dictionary with colours
poplegend_dict = {'0':cl.to_hex('white'), '1':cl.to_hex('blue'), '2':cl.to_hex('purple'), '3':cl.to_hex('cyan'), '4':cl.to_hex('green'), '5':cl.to_hex('yellow'), '6+':cl.to_hex('red')}

Map.add_legend('Legend', boundslegend_dict) #adding legends to Map
Map.add_legend('Population per Cell', poplegend_dict)
Map.add_legend('NO2', no2legend_dict)

for i in range(len(DistrictNamesList)): #Adding District bounds coloured layers to Map
  District_Bounds = Scotland_Bounds.filterMetadata('ADM2_NAME', 'equals', DistrictNamesList[i])
  Map.addLayer(District_Bounds.geometry(), {'color': colors[i]}, DistrictNamesList[i])

Map #Display Map

Below: Plotting Script for generating traditional figures using matplotlib plotting

This script was developed offline (hence the hardcoded arrays)

This script also saves the results to a table in EXCEL (Python library: xlwt)

In [ ]:
#ME975 - Assignment - Jacob Currie - 201718558
#---------------------------------------------
#Plotting/Graphing Script
import matplotlib.pyplot as plt
import numpy as np

#hardcoded arrays of input data taken from google colab variables
DistrictNamesList = ['Borders', 'Central', 'Dumfries & Gal', 'Fyfe', 'Grampian', 'Highland', 'Lothian', 'Orkney', 'Shetland Islands', 'Strathclyde', 'Tayside', 'Western Isles']
Data2019 = [2.2122008554262085e-05,2.3609144718329626e-05,2.2920205749643354e-05,2.996572421056435e-05,1.7438069491396237e-05,1.6123909193907615e-05,
            2.978613519760064e-05,1.2591065901324445e-05,1.2150154708114013e-05,2.2034038841374178e-05,1.916083730769428e-05,1.2717830142645274e-05]
Data2020 = [2.0716044129793837e-05,2.184295374201366e-05,2.0838283314443245e-05,2.7177963032354258e-05,1.8644532261182567e-05,1.5392865174184957e-05,
            2.5521300512970897e-05,1.227162058641417e-05,9.72305618677271e-06,2.04907429503703e-05,1.9815419902427874e-05,1.0717729418158482e-05]
Data2021 = [2.210136053056543e-05,2.207590961163332e-05,2.0215835905095158e-05,3.0077168762210414e-05,1.9320790396829416e-05,1.439016460944674e-05,
            3.158178579266714e-05,1.1731656606566475e-05,1.0305211852537112e-05,2.0276745580832056e-05,1.9620622300899496e-05,1.0706163989725942e-05]
LegendLabels = ['2019', '2020', '2021']            
PopDensity = [2.49289177832715e-05,0.00011463267606779667,2.3413799137766673e-05,0.0002722375960673222,6.604844414118531e-05,8.820244062438711e-06,
            0.0004746136196991526,2.0756991266692525e-05,1.2111503364635487e-05,0.00015828110200027238,5.21235078302237e-05,8.524820130842963e-06]

sortIndex = np.argsort(PopDensity)  #Getting sorted list indices for population density

def argSort(data, indices): #sort list with given list or indices
    newList = []
    for i in indices:
        newList.append(data[i])
    return newList
#Creating sorted lists (SORTED BY ASCENDING POPULATION DENSITY)
PopDensitySorted = argSort(PopDensity, sortIndex)
Data2019Sorted = argSort(Data2019, sortIndex)
Data2020Sorted = argSort(Data2020, sortIndex)
Data2021Sorted = argSort(Data2021, sortIndex)
DistrictsSorted = argSort(DistrictNamesList, sortIndex)

#Saving to Excel doc
import xlwt
W = xlwt.Workbook()
Ws = W.add_sheet("Results")

Ws.write(2,1,"District")
Ws.write(3,1,"2019")
Ws.write(4,1,"2020")
Ws.write(5,1,"2021")
Ws.write(6,1,"Pop Density")

for i in range(0, len(DistrictNamesList)):
    Ws.write(2, i + 2, DistrictNamesList[i])
    Ws.write(3, i + 2, Data2019[i])
    Ws.write(4, i + 2, Data2020[i])
    Ws.write(5, i + 2, Data2021[i])
    Ws.write(6, i + 2, PopDensity[i])

Wa = W.add_sheet("Results2")

Wa.write(1,2,"District")
Wa.write(1,3,"2019")
Wa.write(1,4,"2020")
Wa.write(1,5,"2021")
Wa.write(1,6,"Pop Density")


for i in range(0, len(DistrictNamesList)):
    Wa.write(i + 2,2, DistrictNamesList[i])
    Wa.write(i + 2,3, Data2019[i])
    Wa.write(i + 2,4, Data2020[i])
    Wa.write(i + 2,5, Data2021[i])
    Wa.write(i + 2,6, PopDensity[i])

#W.save("Result.xls")

N = len(DistrictNamesList)  #bar chart spacing
xIndex = np.arange(N) * 4

def NO2_Plot(data, labels): #Bar chart plotting function
    f1, ax = plt.subplots()
    c=['#66c2a5', '#fc8d62', '#8da0cb'] #Colours - colourblind safe and pretty
    for j in range(len(data)):
        ax.bar(xIndex + j, data[j], color=c[j])

    ax.set_xticks(xIndex + 1)
    ax.set_xticklabels(labels)
    ax.legend(LegendLabels)
    ax.set_ylabel('tropospheric NO2 column number density (mol/m^2)')
    ax.set_title('Annual average Tropospheric NO2 Column density per District in Scotland')
    
rawData = [Data2019, Data2020, Data2021] #combine data
NO2_Plot(rawData, DistrictNamesList)    #plot data

rawDataSorted = [Data2019Sorted, Data2020Sorted, Data2021Sorted]  #sorted
NO2_Plot(rawDataSorted, DistrictsSorted)    #sorted plot

def NO2_Line():  #NO2 Line graph Function
    f1, ax = plt.subplots() #colours - not colourblind safe but prettier
    c = ['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f','#ff7f00','#cab2d6','#6a3d9a','#ffff99','#b15928']
    DistrictData = []
    for i in range(len(Data2019)):
        DistrictData.append([Data2019Sorted[i], Data2020Sorted[i], Data2021Sorted[i]])

    for i in range(len(DistrictsSorted)):
        ax.plot([2019, 2020, 2021], DistrictData[i], '-o', color=c[i])

    ax.set_xticks([2019,2020,2021])
    ax.legend(DistrictsSorted, bbox_to_anchor=(1,1), loc="upper left")
    ax.grid()
    ax.set_xlabel('Year')
    ax.set_ylabel('Tropospheric NO2 Column number Density (mol/m^2)')
    ax.set_title('Annual Average Tropospheric NO2 Column density Per Year')

NO2_Line()  #plotting line graph
plt.show()  #show plots